In [2]:
import os
from google.cloud import firestore
from datetime import datetime

In [3]:
def update_status(license_number, bluetooth_id):
    # Set up your Firestore credentials
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "parking-android-app-firebase-adminsdk-tcjkk-9046d67b09.json"

    # Initialize Firestore client
    db = firestore.Client()

    # Reference the users collection
    users_ref = db.collection("users")
    time_ref = db.collection("time")
    time_doc_ref = time_ref.document("timestamp")

    all_users = users_ref.stream()

    for user in all_users:
        user_data = user.to_dict()

        # Iterate through the vehicle maps
        for key, vehicle in user_data.items():
            if isinstance(vehicle, dict) and vehicle.get("license_plate") == license_number and vehicle.get("bluetooth_id").upper() == bluetooth_id.upper():
                user_id = user.id
                if vehicle.get("status") == "Parked":
                    new_status = "Not Parking"
                else:
                    new_status = "Parked"

                # Update the status value in the Firestore database
                users_ref.document(user_id).update({f"{key}.status": new_status})
                
                # Add a timestamp record to the 'time' collection
                current_time = datetime.utcnow()
                time = current_time.strftime("%Y-%m-%dT%H:%M:%S.%fZ")
                time_data = {"time": time, "license_plate": license_number, "status": new_status}
                time_doc_ref.update({"record": firestore.ArrayUnion([time_data])})
                break

In [13]:
# Recognized license plate number
license_number = "2TH4720"
ble_id = "GH:3G:G6:G5:73:GH"

update_status(license_number, ble_id)